In [ ]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', 250)

import matplotlib.pyplot as plt  # To visualize

# set up separate conda environment or other VM to install qgis 3.4.14 
from qgis.core import QgsApplication, QgsVectorLayer, QgsProcessingUtils, QgsProject, QgsVectorFileWriter
from qgis.analysis import QgsNativeAlgorithms
import sys, os

# See https://gis.stackexchange.com/a/155852/4972 for details about the prefix 
QgsApplication.setPrefixPath('/Users/adevoid', True)
qgs = QgsApplication([], False)
qgs.initQgis()

# Append the path where processing plugin can be found
# sys.path.append('/Users/adevoid/Library/Application Support/QGIS/QGIS3/profiles/default/python/plugins')
# /Users/adevoid/Library/Application Support/QGIS/QGIS3/profiles/default

import processing
from processing.core.Processing import Processing

Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())


from shapely.ops import polygonize


empty_border_cor_year = pd.DataFrame(columns=['year','length','CORNO'])


ped_fence_by_corridor_az_buffer = gpd.read_file('../migrant_deaths_analysis/empty_border_by_year/merge_border_wall/all_wall_Arizona.geojson', driver='GeoJSON')
ped_fence_by_corridor_az_buffer=ped_fence_by_corridor_az_buffer.to_crs(epsg=26912)

ped_fence_by_corridor_az_buffer['geometry'] = ped_fence_by_corridor_az_buffer.buffer(300,cap_style=2, join_style=1)

ped_fence_by_corridor_az_buffer.to_file('../migrant_deaths_analysis/empty_border_by_year/scratch/wall_by_corridor_polygon.geojson', driver='GeoJSON')



border = gpd.read_file('../migrant_deaths_analysis/empty_border_by_year/corridors/border_by_corridors.geojson')
border=border.to_crs(epsg=26912)
border = border.rename(columns={"FID": "CORNO"})
border.to_file('../migrant_deaths_analysis/empty_border_by_year/scratch/border_by_corridors1.geojson', driver='GeoJSON')


years = [*range(1990, 2022, 1)]
  
print(years)
  
# this for loop finds the miles of unwalled border each year by polygonoizing pedestrian fencing and walls,
# overlapping them over the border and finding the remaining 
for year in years:
    print(year)
    
    
        
    ped_fence_by_corridor_az_buffer_year=ped_fence_by_corridor_az_buffer[ped_fence_by_corridor_az_buffer['DATE_IN1']<=year]
    
    ped_fence_by_corridor_az_buffer_year.to_file('../migrant_deaths_analysis/empty_border_by_year/scratch/wall_by_corridor_polygon.geojson', driver='GeoJSON')

    
    my_lines = QgsVectorLayer('../migrant_deaths_analysis/empty_border_by_year/scratch/border_by_corridors1.geojson')
    my_polygons = QgsVectorLayer('../migrant_deaths_analysis/empty_border_by_year/scratch/wall_by_corridor_polygon.geojson')






    parameters = {'INPUT' : my_lines, 'OVERLAY' : my_polygons, 'OUTPUT':'memory:buffer'}

    diff = processing.run('qgis:difference', parameters)['OUTPUT']

    print(type(diff))


    # https://gis.stackexchange.com/questions/279652/qgsvectorfilewriter-writeasvectorformat-differents-qgis-2-qgis-3
    # https://gis.stackexchange.com/questions/354517/save-qgsvectorlayer-to-file
    _writer = QgsVectorFileWriter.writeAsVectorFormat(diff,"../migrant_deaths_analysis/empty_border_by_year/scratch/result.geojson","utf-8",diff.crs(),"GeoJSON")

    my_lines_diff_my_polygons = gpd.read_file('../migrant_deaths_analysis/empty_border_by_year/scratch/result.geojson')

    my_lines_diff_my_polygons=my_lines_diff_my_polygons.to_crs(epsg=26912)

    my_lines_diff_my_polygons['length'] = my_lines_diff_my_polygons['geometry'].length


    sum_corridor_length = my_lines_diff_my_polygons.groupby('CORNO')['length'].sum().to_frame().reset_index()
    


    rows_c = []    
        
    for index, row in sum_corridor_length.iterrows():
        rows_c.append([year,row['length'],row['CORNO']])

    
#     cor_cols = [[year,cor_row[0],c_row[0], cor_row[1],cor_row[2],cor_row[3],cor_row[4],cor_row[5],cor_row[6],cor_row[7],cor_row[8],cor_row[9],cor_row[10],cor_row[11]]]
#     print(cor_cols)
    empty_border_cor_year = empty_border_cor_year.append(pd.DataFrame(rows_c,columns=['year','length','CORNO']))
    
empty_border_cor_year['miles']=empty_border_cor_year['length']*0.000621371192    
empty_border_cor_year

empty_border_cor_year.to_csv('../migrant_deaths_analysis/empty_border_by_year/empty_border_cor_year_1990.csv')


/Users/adevoid/opt/anaconda3/envs/pyqgis-dev/lib/python3.7/site-packages/geopandas/_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiLineString'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  aout[:] = out
/Users/adevoid/opt/anaconda3/envs/pyqgis-dev/lib/python3.7/site-packages/geopandas/_vectorized.py:790: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  for geom in data
/Users/adevoid/opt/anaconda3/envs/pyqgis-dev/lib/python3.7/site-packages/geopandas/_vectorized.py:790: DeprecationWarning: 

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
1990
<class 'qgis._core.QgsVectorLayer'>
1991
<class 'qgis._core.QgsVectorLayer'>
1992
<class 'qgis._core.QgsVectorLayer'>
1993
<class 'qgis._core.QgsVectorLayer'>
1994
<class 'qgis._core.QgsVectorLayer'>
1995
<class 'qgis._core.QgsVectorLayer'>
1996
<class 'qgis._core.QgsVectorLayer'>
1997
<class 'qgis._core.QgsVectorLayer'>
1998
<class 'qgis._core.QgsVectorLayer'>
1999
<class 'qgis._core.QgsVectorLayer'>
2000
<class 'qgis._core.QgsVectorLayer'>
2001
<class 'qgis._core.QgsVectorLayer'>
2002
<class 'qgis._core.QgsVectorLayer'>
2003
<class 'qgis._core.QgsVectorLayer'>
2004
<class 'qgis._core.QgsVectorLayer'>
2005
<class 'qgis._core.QgsVectorLayer'>
2006
<class 'qgis._core.QgsVectorLayer'>
2007
<class 'qgis._core.QgsVectorLayer'>
2008
<class 'qgis._core.QgsVectorLayer'>
2009
<class 'qgis._core.QgsV